#Rarefaction, beta-diversity analysis and alpha-diversity calculations

In [1]:
#Import packages
library(phyloseq)
library(vegan)
library(ggplot2)
library(pspearman)
library(OneR)
library(plyr)
library(dplyr)

Loading required package: permute
Loading required package: lattice
This is vegan 2.5-7

Attaching package: ‘dplyr’

The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
#import phyloseq object with 16s count data and metadata
data<-readRDS('statins_16S.rds')

In [3]:
#Rarefy the dataset to even depth
print(min(sample_sums(data)))
rarefied_data=rarefy_even_depth(data, sample.size = min(sample_sums(data)),
  rngseed = 321, replace = FALSE, trimOTUs = TRUE, verbose = TRUE)
#save the rarefied phyloseq object file
saveRDS(rarefied_data,'rarefied_16s.rds')

[1] 25596


`set.seed(321)` was used to initialize repeatable random subsampling.
Please record this for your records so others can reproduce.
Try `set.seed(321); .Random.seed` for the full vector
...
51614OTUs were removed because they are no longer 
present in any sample after random subsampling

...


In [4]:
rarefied_data<-readRDS('rarefied_16s.rds')
rarefied_data

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 37408 taxa and 1512 samples ]
sample_data() Sample Data:       [ 1512 samples by 32 sample variables ]
tax_table()   Taxonomy Table:    [ 37408 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 37408 tips and 35496 internal nodes ]

In [5]:
rarefied_phylum<-tax_glom(rarefied_data,"Phylum")

In [6]:
rarefied_genus<-tax_glom(rarefied_data,"Genus")

In [7]:
saveRDS(rarefied_genus,'rarefied_genus_16S.rds')

In [8]:
rarefied_genus<-readRDS('rarefied_genus_16S.rds')

In [9]:
#Perform beta-diversity analysis on the bray-curtis dissimilarity matrix
set.seed(123)
metadata <- as(sample_data(rarefied_genus), "data.frame")
#calculate distance matrix
bray<-distance(rarefied_genus, method="bray")
#test for association between interindividual gut microbiome variation and statin use, statin*HMG interaction
adonis(bray~ Group*HMG,
       data = metadata,permutations = 3000)
adonis(bray ~ vendor_microbiome+BMI+sex+age+Group*HMG,
       data = metadata,permutations = 3000)


Call:
adonis(formula = bray ~ Group * HMG, data = metadata, permutations = 3000) 

Permutation: free
Number of permutations: 3000

Terms added sequentially (first to last)

            Df SumsOfSqs MeanSqs F.Model      R2    Pr(>F)    
Group        1     0.532 0.53244  3.7513 0.00247 0.0006664 ***
HMG          1     0.426 0.42600  3.0014 0.00198 0.0013329 ** 
Group:HMG    1     0.362 0.36208  2.5510 0.00168 0.0069977 ** 
Residuals 1508   214.040 0.14194         0.99387              
Total     1511   215.361                 1.00000              
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Call:
adonis(formula = bray ~ vendor_microbiome + BMI + sex + age +      Group * HMG, data = metadata, permutations = 3000) 

Permutation: free
Number of permutations: 3000

Terms added sequentially (first to last)

                    Df SumsOfSqs MeanSqs F.Model      R2    Pr(>F)    
vendor_microbiome    1     7.831  7.8308  58.303 0.03636 0.0003332 ***
BMI                  1     2.049  2.0489  15.255 0.00951 0.0003332 ***
sex                  1     1.406  1.4065  10.472 0.00653 0.0003332 ***
age                  1     1.232  1.2315   9.169 0.00572 0.0003332 ***
Group                1     0.453  0.4526   3.370 0.00210 0.0016661 ** 
HMG                  1     0.146  0.1459   1.087 0.00068 0.3458847    
Group:HMG            1     0.239  0.2392   1.781 0.00111 0.0449850 *  
Residuals         1504   202.005  0.1343         0.93799              
Total             1511   215.361                 1.00000              
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [10]:
#Perform beta-diversity analysis on the Weighted Unifrac dissimilarity matrix
set.seed(123)
wunifrac<-distance(rarefied_genus, method="wunifrac")
#test for association between interindividual gut microbiome variation and statin use
adonis(wunifrac~ Group*HMG,
       data = metadata,permutations = 3000)
adonis(wunifrac~ vendor_microbiome+BMI+sex+age+Group*HMG,
       data = metadata,permutations = 3000)


Call:
adonis(formula = wunifrac ~ Group * HMG, data = metadata, permutations = 3000) 

Permutation: free
Number of permutations: 3000

Terms added sequentially (first to last)

            Df SumsOfSqs MeanSqs F.Model      R2   Pr(>F)   
Group        1     0.192 0.19182  2.6108 0.00172 0.030656 * 
HMG          1     0.287 0.28666  3.9017 0.00257 0.005665 **
Group:HMG    1     0.352 0.35227  4.7947 0.00316 0.001333 **
Residuals 1508   110.794 0.07347         0.99256            
Total     1511   111.625                 1.00000            
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Call:
adonis(formula = wunifrac ~ vendor_microbiome + BMI + sex + age +      Group * HMG, data = metadata, permutations = 3000) 

Permutation: free
Number of permutations: 3000

Terms added sequentially (first to last)

                    Df SumsOfSqs MeanSqs F.Model      R2    Pr(>F)    
vendor_microbiome    1     6.288  6.2880  92.177 0.05633 0.0003332 ***
BMI                  1     1.149  1.1493  16.848 0.01030 0.0003332 ***
sex                  1     0.499  0.4986   7.309 0.00447 0.0006664 ***
age                  1     0.673  0.6729   9.865 0.00603 0.0003332 ***
Group                1     0.148  0.1480   2.170 0.00133 0.0646451 .  
HMG                  1     0.052  0.0521   0.763 0.00047 0.5478174    
Group:HMG            1     0.218  0.2177   3.191 0.00195 0.0116628 *  
Residuals         1504   102.598  0.0682         0.91913              
Total             1511   111.625                 1.00000              
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [11]:
rarefied_genotek<-readRDS('rarefied_16s.rds')
#calculate alpha-diversity measures rarefied using ASV count data
richness_df<-estimate_richness(rarefied_genotek,measures=c('Observed','Shannon'))

In [12]:
#add participant id to the df
richness_df$public_client_id<-sample_data(rarefied_genotek)$public_client_id

In [13]:
#export alpha-diversity dataframe for further statistical analysis
write.csv(richness_df,'richness_df.csv')